In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import yfinance as yf
import fredapi as fa
from datetime import date, timedelta
from Modules.Exposure_Report import Exposure
import numpy as np
import matplotlib.pyplot as plt
import Modules.Useful_Functions as u
from statsmodels.tsa.stattools import adfuller
from scipy.stats.mstats import normaltest
import Modules.DataHub as da
from FileFinder import FileFinder as ff
from itertools import combinations

features = da.available_macro_data

feature_combinations = list(combinations(features, 2))

In [97]:


def find_corrs(df, excess, forward, lookback, monthly, combo_of_features):
    
    if excess == "Excess":
        ex = Exposure(df, lookback, forward, list(combo_of_features), "sp_500")
    elif excess == "Straight Returns":
        ex = Exposure(df, lookback, forward, list(combo_of_features))
    
    if monthly == "Monthly":
        bruh = ex.forward_returns_monthly.copy()
    elif monthly == "Daily":
        bruh = ex.forward_returns_daily.copy()
    
    # DIFF_{combo_of_features[0]}_{combo_of_features[1]}
    bruh['Feature_SPREAD'] = bruh[f'{combo_of_features[0]}_scores'] - bruh[f'{combo_of_features[1]}_scores']

    corrs = bruh.corr().drop(index = list(bruh.columns[-3:]))

    top_corrs = {}
    
    #print(corrs)
    
    for col in bruh.columns[-3:]:
        #print(bruh.columns[-3:])
        selected = corrs[col].loc[ abs(corrs[col]) > .77 ]
        
        #selected = selected.drop(index = [i for i in selected.index if i in list(bruh.columns[-3:])])
        
        selected_dict = selected.to_dict()

        if len(selected) > 0:
            # [list(selected_dict.keys())[0]]
            top_corrs[col] = selected_dict

    return top_corrs


In [98]:

of_interest = {}
forwards = np.linspace(5, 252, 10).astype('int32')
#### Still need to add lookbacks
lookbacks = np.linspace(60, 700, 10).astype('int32')
monthly_or_daily = ["Monthly", "Daily"]
excess_or_straight = ["Excess", "Straight Returns"]

for num, feat_combo in enumerate(feature_combinations):
    print(f"{num+1} / {len(feature_combinations)}")

    sector_df = da.get_Dataset('sec', list(feat_combo), "default")

    for excess in excess_or_straight:

        for month in monthly_or_daily:

            for forward in forwards:
                
                for back in lookbacks:
                
                    good_enough = find_corrs(sector_df, excess, int(forward), int(back), month, feat_combo)
                    
                    if good_enough:
                        of_interest[feat_combo] = {}
                        of_interest[feat_combo][excess] = {}
                        of_interest[feat_combo][excess][month] = {}
                        # (feat_combo,excess,month,int(forward))
                        of_interest[feat_combo][excess][month][int(forward)] = {}
                        of_interest[feat_combo][excess][month][int(forward)][int(back)] = good_enough
                    



1 / 91
2 / 91
3 / 91
4 / 91
5 / 91
6 / 91
7 / 91
8 / 91
9 / 91
10 / 91
11 / 91
12 / 91
13 / 91
14 / 91
15 / 91
16 / 91
17 / 91
18 / 91
19 / 91
20 / 91
21 / 91
22 / 91
23 / 91
24 / 91
25 / 91
26 / 91
27 / 91
28 / 91
29 / 91
30 / 91
31 / 91
32 / 91
33 / 91
34 / 91
35 / 91
36 / 91
37 / 91
38 / 91
39 / 91
40 / 91
41 / 91
42 / 91
43 / 91
44 / 91
45 / 91
46 / 91
47 / 91
48 / 91
49 / 91
50 / 91
51 / 91
52 / 91
53 / 91
54 / 91
55 / 91
56 / 91
57 / 91
58 / 91
59 / 91
60 / 91
61 / 91
62 / 91
63 / 91
64 / 91
65 / 91
66 / 91
67 / 91
68 / 91
69 / 91
70 / 91
71 / 91
72 / 91
73 / 91
74 / 91
75 / 91
76 / 91
77 / 91
78 / 91
79 / 91
80 / 91
81 / 91
82 / 91
83 / 91
84 / 91
85 / 91
86 / 91
87 / 91
88 / 91
89 / 91
90 / 91
91 / 91


In [99]:



of_interest




{('nominal_yield',
  'inflation'): {'Straight Returns': {'Monthly': {252: {202: {'Feature_SPREAD': {'Financials': -0.8109317725701074,
       'Energy': -0.7738477884506182}}}}}},
 ('nominal_yield',
  'cpi'): {'Excess': {'Monthly': {169: {273: {'nominal_yield_scores': {'Telecomm': 0.7794502649588229}}}}}},
 ('nominal_yield',
  'real_yield'): {'Straight Returns': {'Monthly': {224: {131: {'Feature_SPREAD': {'Financials': 0.7968243659104934}}}}}},
 ('nominal_yield',
  'yield_curve'): {'Straight Returns': {'Monthly': {197: {273: {'Feature_SPREAD': {'Telecomm': 0.787842159737671}}}}}},
 ('nominal_yield',
  'economic_uncertainty'): {'Straight Returns': {'Monthly': {252: {202: {'Feature_SPREAD': {'Consumer Discretionary': -0.7739498520494221}}}}}},
 ('nominal_yield',
  'market_price'): {'Excess': {'Monthly': {169: {273: {'nominal_yield_scores': {'Telecomm': 0.7794502649588229}}}}}},
 ('nominal_yield',
  'market_volume'): {'Excess': {'Monthly': {169: {273: {'nominal_yield_scores': {'Telecomm': 